In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # noqa
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FactorAnalysis
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import scipy.cluster.hierarchy as sch

In [33]:
df = pd.read_csv('out.csv')
df['Release Date']=pd.to_datetime(df['Release Date'], format = "%m/%d/%Y")


df['Explicit Track']= df['Explicit Track'].astype(bool)
df.drop(['TIDAL Popularity', 'Soundcloud Streams'], axis=1, inplace=True)


df['Spotify Streams']= df['Spotify Streams'].replace(',','', regex=True)
strings_to_nums = df[['All Time Rank','Spotify Playlist Count','Spotify Playlist Reach','YouTube Views','YouTube Likes'
                               ,'TikTok Posts','TikTok Likes','TikTok Views','YouTube Playlist Reach','AirPlay Spins','SiriusXM Spins'
                               ,'Deezer Playlist Reach','Pandora Streams','Pandora Track Stations','Soundcloud Streams','Shazam Counts']].replace(',', '', regex=True)
strings_to_nums = strings_to_nums.apply(pd.to_numeric, args =('coerce','integer'))
df[['All Time Rank','Spotify Playlist Count','Spotify Playlist Reach','YouTube Views','YouTube Likes','TikTok Posts','TikTok Likes'
             ,'TikTok Views','YouTube Playlist Reach','AirPlay Spins','SiriusXM Spins','Deezer Playlist Reach','Pandora Streams','Pandora Track Stations'
             ,'Soundcloud Streams','Shazam Counts']] = strings_to_nums

df['key'] = df['key'].fillna('missing_key')


KeyError: "['Soundcloud Streams'] not in index"

In [34]:
# imputation = ['Spotifoy Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach', 'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach', 'Apple Music Playlist Count', 'AirPlay Spins', 'SiriusXM Spins', 'Deezer Playlist Count', 'Deezer Playlist Reach', 'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations', 'Shazam Counts', 'acousticness', 'danceability', 'duration_ms', 'energy', ]

for (idx, e) in enumerate(df['Spotify ID']):
    if e is np.nan: print(f'{idx}: {df["Track"][idx]}')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Track                       4600 non-null   object        
 1   Album Name                  4600 non-null   object        
 2   Artist                      4595 non-null   object        
 3   Release Date                4600 non-null   datetime64[ns]
 4   ISRC                        4600 non-null   object        
 5   All Time Rank               4600 non-null   object        
 6   Track Score                 4600 non-null   float64       
 7   Spotify Streams             4487 non-null   object        
 8   Spotify Playlist Count      4530 non-null   object        
 9   Spotify Playlist Reach      4528 non-null   object        
 10  Spotify Popularity          3796 non-null   float64       
 11  YouTube Views               4292 non-null   object      